In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Softmax, Dropout
from keras import backend as K
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers

import numpy as np
import numpy.random as random
from nltk.tokenize import word_tokenize

from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
import pandas as pd
import numpy as np 
import math

Using TensorFlow backend.


In [2]:
UNK_TOKEN = '__unk__'
MAX_SEQUENCE_LENGTH = 50
VOCAB_SIZE = 1500




In [3]:
# get fake.csv from kaggle dataset
data_read = pd.read_csv('fake.csv')
data = data_read.sample(frac=1)

split1 = int(math.ceil(len(data)*0.6))
split2 = int(math.ceil(len(data)*0.8))
train = data[:split1]
val = data[split1+1:split2]
test = data[split2+1:]

print(data['type'].unique())
print(len(val))
print(len(train))
print(len(test))

['bias' 'bs' 'conspiracy' 'satire' 'hate' 'fake' 'junksci' 'state']
2599
7800
2598


# Preprocessing

In [4]:
train_label_str, train_data = train['type'].values.tolist(), train['title'].values.tolist()
val_label_str, val_data = val['type'].values.tolist(), val['title'].values.tolist()
test_label_str, test_data = test['type'].values.tolist(), test['title'].values.tolist()
type_d = data['type'].unique()
myclass = []
mylabels = {}
emotions = {
	"bias":0,
	"conspiracy":1,
	"fake":2,
	"bs":3,
	"satire":4,
	"hate":5,
	"junksci":6,
	"state":7
}
for mytype in data['type'].values.tolist():
    for key,value in emotions.items():
        if mytype == key:
            myclass.append(value)
            mylabels[value] = key
train_labels = myclass[:split1]
val_labels = myclass[split1+1:split2]
test_labels = myclass[split2+1:]
len(emotions)
print(len(train_labels))

7800


In [5]:

# Preprocess data
def preprocess(data):
    # Tokenize (separate by space, punctuation etc) and lowercase all data
    return [word_tokenize(str(t).decode('utf-8').strip().lower()) for t in data]

train_data = preprocess(train_data)
val_data = preprocess(val_data)
test_data = preprocess(test_data)
print(train_data[:1])

[[u'kenyan', u'refugee', u'kills', u'co-worker', u',', u'self']]


In [6]:
# Prepare vocabulary
full_vocab = dict()
for instance in train_data:
    for token in instance:
        full_vocab[token] = 1 + full_vocab.get(token, 0)

# Sort vocabulary by occurence
sorted_vocab = sorted(full_vocab.keys(), key=lambda word: -full_vocab[word])

# Print some samples
print("Vocabulary size: %d"%(len(sorted_vocab)))
print("Most frequent tokens")
for i in range(10):
    print("\t%s: %d"%(sorted_vocab[i], full_vocab[sorted_vocab[i]]))
print("Least frequent tokens")
for i in range(1,11):
    print("\t%s: %d"%(sorted_vocab[-i], full_vocab[sorted_vocab[-i]]))

# We can choose to limit the vocab_size here to only a portion of the original vocab,
# i.e. ignore infrequent tokens to save on memory
vocab_size = VOCAB_SIZE
    
# Create final vocab
word2idx = {w: idx for idx, w in enumerate(sorted_vocab[:vocab_size])}
idx2word = {idx: w for idx, w in enumerate(sorted_vocab[:vocab_size])}


word2idx[UNK_TOKEN] = vocab_size
idx2word[vocab_size] = UNK_TOKEN
vocab_size = vocab_size + 1

Vocabulary size: 14331
Most frequent tokens
	the: 2546
	:: 1928
	to: 1924
	’: 1843
	,: 1448
	of: 1412
	in: 1201
	trump: 1052
	s: 952
	a: 926
Least frequent tokens
	corey: 1
	irony: 1
	yes…: 1
	11/4/16: 1
	zenit: 1
	latakia: 1
	الكويتي: 1
	google/amazon: 1
	lui-même: 1
	cliff: 1


In [7]:
train_data = [[t if t in word2idx else UNK_TOKEN for t in instance] for instance in train_data]
val_data = [[t if t in word2idx else UNK_TOKEN for t in instance] for instance in val_data]
test_data = [[t if t in word2idx else UNK_TOKEN for t in instance] for instance in test_data]

print("Number of tokens filtered out as unknown:")
print("Train: %d/%d"%(len([1 for instance in train_data for t in instance if t == UNK_TOKEN]), sum([len(i) for i in train_data])))
print("val: %d/%d"%(len([1 for instance in val_data for t in instance if t == UNK_TOKEN]), sum([len(i) for i in val_data])))
print("Test: %d/%d"%(len([1 for instance in test_data for t in instance if t == UNK_TOKEN]), sum([len(i) for i in test_data])))

Number of tokens filtered out as unknown:
Train: 23892/92733
val: 8563/31434
Test: 8620/31172


In [8]:
## data_to_tensor
# Given a list of instances, where each instance is a list of tokens,
# this function does the following:
# 1: Replace each token with its corresponding index
# 2: Pad sequences to MAX_SEQUENCE_LENGTH (or truncate them if longer)
#       Padding is done with a unique element, in this case `vocab_size`
#       The network will learn that this unique element is padding and does not
#        mean anything semantically
# 3: Package everything nicely as a NUM_INSTANCES x MAX_SEQUENCE_LENGTH matrix
def data_to_tensor(data):
    # First convert from words to indices
    idx_data = [[word2idx[t] for t in instance] for instance in data]
    
    # Create numpy representation
    return pad_sequences([np.array(d) for d in idx_data], maxlen=MAX_SEQUENCE_LENGTH, dtype='int32', padding='pre', truncating='pre', value=vocab_size)

X_train = data_to_tensor(train_data)
y_train = to_categorical(np.array(train_labels, dtype=np.int))
print(np.unique(train_labels))

X_val = data_to_tensor(val_data)
y_val = to_categorical(np.array(val_labels, dtype=np.int))
print(np.unique(val_labels))
X_test = data_to_tensor(test_data)
y_test = to_categorical(np.array(test_labels, dtype=np.int))
print(np.unique(test_labels))

vocab_size = vocab_size + 1 # Add 1 for the padding token
print(vocab_size)

[0 1 2 3 4 5 6 7]
[0 1 2 3 4 5 6 7]
[0 1 3 4 5 6 7]
1502


In [9]:
def idx_to_bow(data):
    data_bow = np.zeros((data.shape[0], vocab_size)) 
    for i in range(data.shape[1]):
        data_bow[np.arange(data.shape[0]), data[:, i]] = 1
    return data_bow

X_train_bow = idx_to_bow(X_train)
X_val_bow = idx_to_bow(X_val)
X_test_bow = idx_to_bow(X_test)
print(len(emotions))

8


In [10]:
model = Sequential()
model.add(Dense(25, input_shape=(vocab_size,), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.25))
model.add(Dense(len(emotions)))
model.add(Softmax())
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 25)                37575     
_________________________________________________________________
dropout_1 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 208       
_________________________________________________________________
softmax_1 (Softmax)          (None, 8)                 0         
Total params: 37,783
Trainable params: 37,783
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(X_train_bow, y_train, epochs=100, validation_data=(X_val_bow, y_val))

Train on 7800 samples, validate on 2599 samples
Epoch 1/100
7800/7800 [==============================] - 1s 87us/step - loss: 0.5482 - acc: 0.8853 - val_loss: 0.5161 - val_acc: 0.8888
Epoch 2/100
7800/7800 [==============================] - 1s 79us/step - loss: 0.5448 - acc: 0.8847 - val_loss: 0.5148 - val_acc: 0.8888
Epoch 3/100
7800/7800 [==============================] - 1s 77us/step - loss: 0.5444 - acc: 0.8849 - val_loss: 0.5134 - val_acc: 0.8896
Epoch 4/100
7800/7800 [==============================] - 1s 78us/step - loss: 0.5414 - acc: 0.8851 - val_loss: 0.5097 - val_acc: 0.8892
Epoch 5/100
7800/7800 [==============================] - 1s 87us/step - loss: 0.5352 - acc: 0.8859 - val_loss: 0.5091 - val_acc: 0.8903
Epoch 6/100
7800/7800 [==============================] - 1s 92us/step - loss: 0.5354 - acc: 0.8862 - val_loss: 0.5098 - val_acc: 0.8896
Epoch 7/100
7800/7800 [==============================] - 1s 80us/step - loss: 0.5339 - acc: 0.8865 - val_loss: 0.5092 - val_acc: 0.8888


7800/7800 [==============================] - 1s 103us/step - loss: 0.4463 - acc: 0.8959 - val_loss: 0.4942 - val_acc: 0.8961
Epoch 61/100
7800/7800 [==============================] - 1s 95us/step - loss: 0.4429 - acc: 0.8967 - val_loss: 0.4955 - val_acc: 0.8957
Epoch 62/100
7800/7800 [==============================] - 1s 95us/step - loss: 0.4451 - acc: 0.8963 - val_loss: 0.4973 - val_acc: 0.8969
Epoch 63/100
7800/7800 [==============================] - 1s 91us/step - loss: 0.4389 - acc: 0.8968 - val_loss: 0.4951 - val_acc: 0.8969
Epoch 64/100
7800/7800 [==============================] - 1s 98us/step - loss: 0.4380 - acc: 0.8960 - val_loss: 0.4951 - val_acc: 0.8977
Epoch 65/100
7800/7800 [==============================] - 1s 91us/step - loss: 0.4406 - acc: 0.8955 - val_loss: 0.4921 - val_acc: 0.8984
Epoch 66/100
7800/7800 [==============================] - 1s 97us/step - loss: 0.4368 - acc: 0.8982 - val_loss: 0.4948 - val_acc: 0.8980
Epoch 67/100
7800/7800 [=============================